Video of Volcano Camera Images During Certain Hours of Each Day
--

This focuses on a sub-area of the image and only images recorded during certain hours of each day.

In [61]:
import cv2
import matplotlib.pyplot as plt
import os
import datetime
from datetime import timedelta 
import pytz
import pandas as pd

%matplotlib inline

In [62]:
def imfdt(imfile):
    #gets a datetime for an image file named like 20180101001002WIWR.jpg
    year = imfile[:4]
    month = imfile[4:6]
    day = imfile[6:8]
    hour = imfile[8:10]
    minute = imfile[10:12]
    second = imfile[12:14]
    dt = pytz.utc.localize(datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second)))
    return (dt)

In [63]:
def clahe(img):
    # CLAHE (Contrast Limited Adaptive Histogram Equalization)
    #https://stackoverflow.com/questions/19363293/whats-the-fastest-way-to-increase-color-image-contrast-with-opencv-in-python-c
    global img2
    clahe = cv2.createCLAHE(clipLimit=3., tileGridSize=(8,8))

    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # convert from BGR to LAB color space
    l, a, b = cv2.split(lab)  # split on 3 different channels

    l2 = clahe.apply(l)  # apply CLAHE to the L-channel

    lab = cv2.merge((l2,a,b))  # merge channels
    img2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BGR

**Set image date range and camera**

In [64]:
#UTC date range for images to view
date1 = '2017-12-01'
date2 = '2017-12-31'
camera = 'WIWR'

**Text for image label**

In [65]:
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,30)
fontScale              = 0.5
fontColor              = (255,255,255)
lineType               = 1

**Set hours for image selection**

In [66]:
hour1 = 0
hour2 = 2
strhour1 = str(hour1).zfill(2)
strhour2 = str(hour2).zfill(2)

**Relevant directories**

In [67]:
#as mounted, this is image directory
camdir = '/run/user/1000/gvfs/ftp:host=ftp.geonet.org.nz/volcano_cams'

#directory to write crops
imgdir = '/home/sherburn/geonet/volcam_images'
crpdir = os.path.join(basedir, 'crop')
viddir = os.path.join(basedir, 'video')

**Loop through date folders**

In [68]:
for date in pd.date_range(date1, date2):
    year = date.year
    time_start = pytz.utc.localize(date + timedelta(hours=hour1))
    time_end = pytz.utc.localize(date + timedelta(hours=hour2))
    datestr = date.strftime('%Y%m%d')
    print (datestr)
    dydir = os.path.join(camdir, str(year), datestr, camera)
    
    for file in sorted(os.listdir(dydir)):
        if file.endswith('.jpg'):
            imfile = os.path.join(dydir, file)
            imtime = imfdt(file)
            if (imtime < time_end and imtime > time_start):
                img = cv2.imread(imfile)
                crop = img[400:700,750:1450]
                clahe(crop) #enhanced the crop
                cv2.putText(img2,imtime.strftime('%Y-%m-%d %H:%M'), bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
                crfile = os.path.join(crpdir, file)
                cv2.imwrite(crfile, img2)
height,width,layers = crop.shape

20171201
20171202
20171203
20171204
20171205
20171206
20171207
20171208
20171209
20171210
20171211
20171212
20171213
20171214
20171215
20171216
20171217
20171218
20171219
20171220
20171221
20171222
20171223
20171224
20171225
20171226
20171227
20171228
20171229
20171230
20171231


**Make video of cropped images**

In [69]:
vidname = camera+'_'+date1+'_'+date2+'_'+strhour1+'-'+strhour2+'.avi'
vidfile = os.path.join(viddir, vidname)

fps = 6
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(vidfile,fourcc,fps,(width,height))
for file in sorted(os.listdir(crpdir)):
    if file.endswith('.jpg'):
        imfile = os.path.join(crpdir, file)
        img = cv2.imread(imfile)
        video.write(img)
cv2.destroyAllWindows
video.release()